# Beginner data cleaning tutorial
## Most of the information comes from the pandas kaggle course
> [Kaggle data cleaning course](https://www.kaggle.com/learn/data-cleaning)

> **data** is stored in ./data/NFL-play-by-play
***

In [7]:
# preparation
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 10)

In [9]:
# load dataset
nfl_df = pd.read_csv('./data/NFL-play-by-play/NFL Play by Play 2009-2017 (v4).csv', low_memory=False)

## Handling Missing Values

The first thing we'll need to do is load in the libraries and dataset we'll be using.

**The first thing** to do when you get a new dataset **is take a look at some of it**. This lets you see that it all read in correctly and gives an idea of what's going on with the data. In this case, let's see if there are any missing values, which will be reprsented with NaN or None.

In [10]:
nfl_df.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


### How many missing data points do we have?

In [13]:
miss_data_count = nfl_df.isnull().sum()
miss_data_count.iloc[:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

That seems like a lot! **It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem**

In [26]:
# it seems like 
## total_cells = np.product(nfl_df.shape())
total_cells = nfl_df.size

# it seems like
## miss_data_count.sum()
miss_cells = miss_data_count.sum()

percent_missing = (miss_cells / total_cells) * 100

print(f"percent of missing data is {percent_missing}")

percent of missing data is 24.87214126835169


Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

### Figure out why the data is missing (**imputation**)

For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important questions you can ask yourself to help figure this out is this:

> **Is this value missing because it wasn't recorded or because it doesn't exist?**



If **a value is missing becuase it doesn't exist** (like the height of the oldest child of someone who doesn't have any children) then **it doesn't make sense to try and guess what it might be**. These values you probably do want to keep as NaN

if **a value is missing because it wasn't recorded**, then you can **try to guess what it might have been based on the other values in that column and row**.

This is called **imputation**

In [36]:
# # look at the # of missing points in the first ten columns
nfl_df.isnull().sum().iloc[:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

By looking at the **documentation**, You can see that this **column has information on the number of seconds left in the game when the play was made**. **This means that these values are probably missing because they were not recorded**, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, **there are other fields, like "PenalizedTeam" that also have lot of missing fields**. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say which team was penalized. **For this column**, it would make more sense to either **leave it empty or to add a third value like "neither" and use that to replace the NA's**.

> **Tip**: This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.



### Drop missing values

If you're in a hurry or don't have a reason to figure out why your values are missing**, one option you have is to just remove any rows or columns that contain missing values. **(Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)**

If you're sure you want to drop rows with missing values, pandas does have a handy function, dropna() to help you do this. Let's try it out on our NFL dataset!

In [38]:
# remove all the rows that contain a missing value
nfl_df.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Oops, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. 

In [45]:
# trying to do it for columns
clean_nfl_df = nfl_df.dropna(axis=1).head()

let's calculate how much data did we lose?

In [47]:
print("Columns in df: {0}".format(nfl_df.shape[1]))
print("Columns in df without NA: {0}".format(clean_nfl_df.shape[1]))

Columns in df: 102
Columns in df without NA: 41


We've lost quite a bit of data, but at this point we have successfully removed all the NaN's from our data.

### Filling in missing values automatically 

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [49]:
subset_nfl_data = nfl_df.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


We can use the Panda's `fillna()` function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [50]:
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [51]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the remaining na's with 0
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009
